<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# NLCD downsampling and reprojection

_by Jeremy Freeman (CarbonPlan), August 2, 2020_

This notebook downsamples and reprojects NLCD 30m rasters stored in Cloud Optimized GeoTIFF
into 30m, 250m, and 4000m GeoTIFFs.

**Inputs:**

- COG outputs from `01_nlcd_to_cogs.ipynb`

**Outputs:**

- COG outputs after downsampling and reprojection

**Notes:**

- Source CRS and projection extent come from NLCD

In [1]:
import os

In [2]:
from utils import setup, projections
workdir, upload = setup('jeremy')

In [3]:
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
dst_profile = cog_profiles.get('deflate')

In [4]:
def get_file(region, year, resolution): 
    return {
        'source': workdir / f'nlcd/{region}/30m/{year}.tif',
        'target': f'processed/nlcd/{region}/{resolution}m/{year}.tif',
    }

In [7]:
for resolution in [30]:
    for region in ['ak', 'conus']:
        if region == 'ak':
            years = [2011, 2016]
        if region == 'conus':
            years = [2001, 2004, 2006, 2008, 2011, 2013, 2016]
        for year in years:
            f = get_file(region, year, resolution)
            crs, extent = projections('albers', region)
            if resolution == 30:
                resampling = 'near'
            else:
                resampling = 'mode'
            cmd = (
            'gdalwarp '
            "-t_srs '%s' "
            '-te %s '
            '-tr %s %s '
            '-r %s '
            '%s '
            '%s') % (crs, extent, resolution, resolution, resampling, f['source'], './raster.tif')
            #os.system(cmd)
            #cog_translate('./raster.tif', './raster.tif', dst_profile)
            upload(f['source'], f['target'])
            #os.remove('./raster.tif')

ReadTimeout: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Read timed out. (read timeout=60)